In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("./HotspotAnalysis/")
import os
from AnalysisFunctions import *
from ROIFunctions import *
from PlottingFunctions import *
import cv2
import numpy as np
from imageio import imwrite
import holoviews as hv
from holoviews import opts
from skimage.measure import block_reduce
from IPython.display import display
hv.notebook_extension('bokeh')

from scipy.stats import skew
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from tqdm.notebook import tqdm
import pingouin as pg

# from IPython.display import display, HTML
# display(HTML("""
# <style>
# .output {
#    display: flex;
#    align-items: center;
#    text-align: center;
#    }
# </style>
# """))
# display(HTML("<style>.container { width:90% !important; }</style>"))

import hotspot_util
fig_config = {'toImageButtonOptions':{'format': 'svg','scale':2}}

## Load an image

In [ ]:
imgPath = '/Users/joezaki/Documents/Bentley_Lab/Mesias_2023/Revision_Analyses/Hotspot_Validation_Analysis/Hotspot_Validation_Data/tiff/'
images = np.array(os.listdir(imgPath))
image_bool = [x.split('.')[-1] == 'tiff' for x in images]
not_roi_bool = ['ROI' not in x for x in images]
images = images[(image_bool and not_roi_bool)]

imgName = images[5]
savePath = None

img, DAPI = loadImage(imgPath=imgPath, imgName=imgName, img_channel=0, DAPI_channel=1)

name = imgName.split(".")[0]
opts.defaults(
    opts.GridSpace(shared_xaxis=True, shared_yaxis=True),
    opts.Image(cmap='gist_gray', width=int(img.shape[1]/3), height=int(img.shape[0]/3)),
    opts.Labels(text_color='white', text_font_size='8pt', text_align='left', text_baseline='bottom'),
    opts.Path(color='white'),
    opts.Spread(width=600),
    opts.Overlay(show_legend=False))

%output size=80
allChannels = np.dstack((DAPI, img))
ds = hv.Dataset((['DAPI','Image'], np.arange(DAPI.shape[1]), np.arange(DAPI.shape[0]), allChannels), ['Channel','X', 'Y'], 'Intensity')
allChannelsPlot = ds.to(hv.Image, ['X', 'Y'], dynamic=True)
allChannelsPlot.opts(invert_yaxis=True)

## Remove saturated pixels

In [ ]:
#%%output size=100
if type(img[0,0]) == np.uint8:
    threshold = 200
elif type(img[0,0]) == np.uint16:
    threshold = 50000
else:
    raise Exception("img is not 8- or 16-bit")
imgNew = img.copy()
avgPixel = np.mean(img)
imgNew[imgNew > threshold] = avgPixel
thresholded = np.dstack((img, imgNew))
ds = hv.Dataset((np.arange(2), np.arange(img.shape[1]), np.arange(img.shape[0]), thresholded), ['Threshold','X', 'Y'], 'Intensity')
thresholdedPlots = ds.to(hv.Image, ['X', 'Y'], dynamic=True)
thresholdedPlots.opts(invert_yaxis=True)

In [ ]:
img = imgNew

In [ ]:
roiPath = os.path.join(imgPath, '{}_ROI.tiff'.format(imgName.split('.')[0]))
mask, croppedImg, maskedImage, ROIimage, processedImage = load_ROI(img, DAPI, roiPath)
#del DAPI

## Create processed image of randomized pixels

In [ ]:
randArray = maskedImage.copy()
random_pixels = np.random.choice(a=maskedImage[~maskedImage.mask].data, size=maskedImage[~maskedImage.mask].data.size, replace=False)
randArray[~randArray.mask] = random_pixels

In [ ]:
# from tqdm import tqdm_notebook
# for i, val in enumerate(tqdm_notebook(random_pixels)):
#     rand_array[~rand_array.mask][i] = val

In [ ]:
maskedImage.shape

In [ ]:
%output size=25
allChannels = np.dstack((maskedImage.filled(-1), randArray.filled(-1)))
ds = hv.Dataset((['Image','Randomized'], np.arange(maskedImage.shape[1]), np.arange(maskedImage.shape[0]), allChannels), ['Channel','X', 'Y'], 'Intensity')
allChannelsPlot = ds.to(hv.Image, ['X', 'Y'], dynamic=True)
allChannelsPlot.opts(opts.Image(tools=['hover'], invert_yaxis=True, width=int(maskedImage.shape[1]), height=int(maskedImage.shape[0])))

## Run hotspot analysis on both actual image and randomized image

In [ ]:
FULLname = 'P56_RBP4Cre_Dual_Inj_050320_sectionB5R_20x20'

In [ ]:
%%time
nx = 20
ny = 20
FULLname = name + "_" + str(nx) + "x" + str(ny)
# stats_actual = Getis(mask, maskedImage, nx, ny)

stats_actual = Getis_parallel(mask, maskedImage, nx, ny)

direction_actual, zs_actual, DL_actual, VL_actual, DM_actual, VM_actual, MLaxisZs_actual, DVaxisZs_actual, quadrantStds_actual = processedStats(stats_actual, maskedImage, FULLname)

In [ ]:
%%time
nx = 20
ny = 20
FULLname = name + "_" + str(nx) + "x" + str(ny)
# stats_rand = Getis(mask, randArray, nx, ny)

stats_rand = Getis_parallel(mask, randArray, nx, ny)

direction_rand, zs_rand, DL_rand, VL_rand, DM_rand, VM_rand, MLaxisZs_rand, DVaxisZs_rand, quadrantStds_rand = processedStats(stats_rand, randArray, FULLname)

### Save actual and random image stats

In [ ]:
if savePath is None:
    savePath = imgPath
    savePath = os.path.abspath(savePath)

path = os.path.join(savePath, FULLname)
try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)
os.chdir(path)

stats_actual.to_csv((FULLname + "_GetisOrdStats_Actual.csv"))
stats_rand.to_csv((FULLname + "_GetisOrdStats_Randomized.csv"))

***
### Calculate how the mean, sd, and skew of the z-scores varies depending on the neighborhood size

In [ ]:
max_n_size = np.round((np.max(img.shape)/5)/20)*20
delta_n = 10
neighborhood_sizes = np.arange(delta_n, max_n_size, delta_n).astype(int)

neighborhood_size_df = pd.DataFrame(columns=['Neighborhood', 'Mean', 'SD', 'Skew'])

for n_size in tqdm(neighborhood_sizes):
    nx = n_size
    ny = n_size
    
    neighborhood_stats = Getis_parallel(mask, maskedImage, nx, ny)
    zscores = neighborhood_stats['Z-Score']
    neighborhood_size_df = neighborhood_size_df.append(pd.Series({'Neighborhood':n_size, 'Mean':zscores.mean(), 'SD':zscores.std(), 'Skew':skew(zscores)}), ignore_index=True) 

In [ ]:
fig = make_subplots(cols=3)

for i, readout in enumerate(['Mean', 'SD', 'Skew']):
    fig.add_trace(go.Scattergl(x=neighborhood_size_df['Neighborhood'], y=neighborhood_size_df[readout],
                               mode='lines+markers', line=dict(width=2, color='black'), showlegend=False), row=1, col=i+1)
    fig.update_xaxes(title_text='Neighborhood Size')
    y_range = (0,neighborhood_size_df[readout].max()+1)
    y_range = None
    fig.update_yaxes(title_text=readout, range=y_range, row=1, col=i+1)

fig.update_layout(template='simple_white', width=1200, height=400)
fig.show()

***
### Plot actual vs random hotspots and distributions (optional), and save if desired

In [ ]:
%output size=100
scale_rand_plot = True
allChannels = np.dstack((zs_actual, zs_rand))
ds = hv.Dataset((['Image','Randomized'], np.arange(zs_actual.shape[1]), np.arange(zs_actual.shape[0]), allChannels), ['Channel','X', 'Y'], 'Intensity')
allChannelsPlot = ds.to(hv.Image, ['X', 'Y'], dynamic=True)
if scale_rand_plot:
    display(allChannelsPlot.opts(opts.Image(tools=['hover'],
                                            cmap='Spectral_r', colorbar=True, invert_yaxis=True,
                                            width=int(maskedImage.shape[1]), height=int(maskedImage.shape[0]),
                                            clim=(zs_actual[~np.isnan(zs_actual)].min(),zs_actual[~np.isnan(zs_actual)].max()))))
else:
    display(allChannelsPlot.opts(opts.Image(tools=['hover'],
                                            width=int(maskedImage.shape[1]), height=int(maskedImage.shape[0]),
                                            cmap='Spectral_r', colorbar=True, invert_yaxis=True)))

In [ ]:
Heatmap_actual = HeatmapPlot(zs_actual)
Heatmap_rand = HeatmapPlot(zs_rand)
# Heatmap_actual.savefig(((FULLname + "actual_Heatmap_Plot.pdf")), bbox_inches="tight")
# Heatmap_rand.savefig(((FULLname + "rand_Heatmap_Plot.pdf")), bbox_inches="tight")

In [ ]:
freq_actual, edges_actual = np.histogram(zs_actual[~np.isnan(zs_actual)], 100)
freq_rand, edges_rand     = np.histogram(zs_rand[~np.isnan(zs_rand)], 10)

color1 = 'steelblue'
color2 = 'darkred'
xlabel = 'Z Scores'
alpha = 0.8
line_width = 0.1
plot_width = 800
aspect = 2

same_plot = True
if same_plot:
    fig = hv.NdOverlay({'Image': hv.Histogram((edges_actual, freq_actual)).opts(color=color1),
                        'Randomized': hv.Histogram((edges_rand, freq_rand)).opts(color=color2)}).opts(
        'Histogram', xlabel=xlabel, width=plot_width, aspect=aspect, alpha=alpha, line_width=line_width, title='Image & Randomized Image')
    fig.get_dimension('Element').label=''
    display(fig)
else:
    display(
        hv.Histogram((edges_actual, freq_actual)).opts(xlabel=xlabel, width=plot_width, aspect=aspect, color=color1, alpha=alpha, line_width=line_width).opts(title='Image') +
        hv.Histogram((edges_rand, freq_rand)).opts(xlabel=xlabel, width=plot_width, aspect=aspect, color=color2, alpha=alpha, line_width=line_width).opts(title='Randomized Image'))

In [ ]:
DV_ML_actual = DV_ML_Plot(stats_actual, zs_actual, MLaxisZs_actual, DVaxisZs_actual, direction_actual)
DV_ML_rand = DV_ML_Plot(stats_rand, zs_rand, MLaxisZs_rand, DVaxisZs_rand, direction_rand)
# DV_ML_actual.savefig(((FULLname + "actual_DV_ML_Plot.pdf")), bbox_inches="tight")
# DV_ML_rand.savefig(((FULLname + "rand_DV_ML_Plot.pdf")), bbox_inches="tight")

In [ ]:
Quadrant_actual = QuadrantPlot(stats_actual, maskedImage, DL_actual, VL_actual, DM_actual, VM_actual)
Quadrant_rand = QuadrantPlot(stats_rand, randArray, DL_rand, VL_rand, DM_rand, VM_rand)
# Quadrant_actual.savefig(((FULLname + "actual_Quadrant_Plot.pdf")), bbox_inches="tight")
# Quadrant_rand.savefig(((FULLname + "rand_Quadrant_Plot.pdf")), bbox_inches="tight")

***
### Measure summary stats of actual vs random images for a list of images

In [ ]:
overall_path = '/Users/joezaki/Documents/Bentley_Lab/Mesias_2023/Revision_Analyses/Hotspot_Validation_Analysis/Hotspot_Validation_Data/tiff/'

file_list = ['S1_PFC_Adult1b_sectionB4R', 'S1PFC_Adult1c_sectionB4R', 'P56_307GFP_sectionA1L', 'P56_RBP4Cre_Dual_Inj_050320_sectionB5R']

summary_df = pd.DataFrame(columns=['Filename', 'ActualRand', 'Mean', 'SD', 'Skew'])

for file in file_list:
    actual_path = os.path.join(overall_path, '{}_20x20/{}_20x20_GetisOrdStats_Actual.csv'.format(file,file))
    rand_path = os.path.join(overall_path, '{}_20x20/{}_20x20_GetisOrdStats_Randomized.csv'.format(file,file))

    actual_df = pd.read_csv(actual_path)
    rand_df = pd.read_csv(rand_path)

    fig = ff.create_distplot(hist_data=[actual_df['Z-Score'].values, rand_df['Z-Score'].values], group_labels=['Actual', 'Randomized'],
                             show_rug=False, bin_size=1, colors=['steelblue', 'darkred'])
    fig.update_layout(template='simple_white', width=600, height=400, font=dict(size=15),
                      xaxis_title='Z-Score', yaxis_title='Frequency', title_text=file)
    fig.update_yaxes(type='log', tickformat='.0e')
    fig.show(renderer='notebook_connected', config=fig_config)

    for label, df in zip(['Actual','Randomized'], [actual_df, rand_df]):
        mean_z = df['Z-Score'].mean()
        sd_z   = df['Z-Score'].std()
        skew_z = skew(df['Z-Score'])
        summary_df = summary_df.append(pd.Series({'Filename':file, 'ActualRand':label, 'Mean':mean_z, 'SD':sd_z, 'Skew':skew_z}), ignore_index=True)

In [ ]:
agg_data = summary_df.copy()
groupby = 'ActualRand'
plot_var = 'Mean'
hotspot_util.plotMeanData(agg_data=agg_data, groupby=groupby, plot_var=plot_var, plot_datalines=True, colors = ['steelblue', 'darkred'], y_title=plot_var, y_range=(0,agg_data[plot_var].max()+0.1), plot_width=300)
pg.rm_anova(data=agg_data, dv=plot_var, within='ActualRand', subject='Filename')